# BirdCLEF 2023 - Audio Preprocessing
The goal here is to apply the guidelines and approach presented in the [First Steps, Road to the Top](https://www.kaggle.com/code/jhoward/first-steps-road-to-the-top-part-1) series of notebooks. Following that series, we'll look to use FastAI/PyTorch to create an effective validation set and iterate rapidly!

To overall plan is to start with transfer learning to train CNNs on spectrograms.

This notebook focuses on one-time audio preprocessing that will take place prior to creation of data loaders.

## Todo

 1. [ ] Complete Part 1
 2. [ ] Complete Part 2
 3. [ ] Complete Part 3
 4. [ ] Complete Part 4
 5. [ ] Try fastaudio for audio preprocessing [fastaudio]
 6. [ ] Try TorchAudio for audio preprocessing [torchaudio]
 7. [ ] Experiment with secondary labels

# Setup

## Download competition data and install packages
Prerequisites if running off of Kaggle:
1. Create a python environment
2. [Create `~/.kaggle/kaggle.json`](https://www.kaggle.com/docs/api)
3. [Join the competition](https://www.kaggle.com/competitions/birdclef-2023)

In [1]:
from fastkaggle import setup_comp

comp  = 'birdclef-2023'
comp_path = setup_comp(comp, install='fastai timm fastaudio')
comp_path.ls()

(#8) [Path('birdclef-2023/train_audio_prep-2'),Path('birdclef-2023/eBird_Taxonomy_v2021.csv'),Path('birdclef-2023/test_soundscapes'),Path('birdclef-2023/train_metadata.csv'),Path('birdclef-2023/train_audio_prep-2b'),Path('birdclef-2023/train_audio'),Path('birdclef-2023/sample_submission.csv'),Path('birdclef-2023/train_audio_prep-1')]

## Load packages

In [2]:
from fastai.vision.all import *
import pandas as pd
import itertools
import torch
import torchaudio
import IPython.display
import mimetypes
from tqdm.auto import tqdm

from sklearn.model_selection import StratifiedKFold
from fastaudio.core.all import *
from fastaudio.augment.all import *

print(torch.__version__, torchaudio.__version__)

/home/sdhca/mambaforge/envs/fastai/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


2.0.0+cu117 2.0.1+cu117


# Load Metadata

In [3]:
trn_path = comp_path/'train_audio'
trn_path.ls()

(#264) [Path('birdclef-2023/train_audio/somgre1'),Path('birdclef-2023/train_audio/whcpri2'),Path('birdclef-2023/train_audio/egygoo'),Path('birdclef-2023/train_audio/woosan'),Path('birdclef-2023/train_audio/yebapa1'),Path('birdclef-2023/train_audio/hamerk1'),Path('birdclef-2023/train_audio/hartur1'),Path('birdclef-2023/train_audio/rebfir2'),Path('birdclef-2023/train_audio/strher'),Path('birdclef-2023/train_audio/wtbeat1')...]

## Provided Metadata

In [4]:
trn_df = pd.read_csv(comp_path/'train_metadata.csv')
lst_cols = ['secondary_labels', 'type']
for col in lst_cols:
    trn_df[col] = trn_df[col].apply(lambda x: x[1:-1].split(','))
    
trn_df.head().T

,0,1,2,3,4
primary_label,abethr1,abethr1,abethr1,abethr1,abethr1
secondary_labels,[],[],[],[],[]
type,['song'],['call'],['song'],['song'],"['call', 'song']"
latitude,4.3906,-2.9524,-2.9524,-2.9524,-2.9524
longitude,38.2788,38.2921,38.2921,38.2921,38.2921
scientific_name,Turdus tephronotus,Turdus tephronotus,Turdus tephronotus,Turdus tephronotus,Turdus tephronotus
common_name,African Bare-eyed Thrush,African Bare-eyed Thrush,African Bare-eyed Thrush,African Bare-eyed Thrush,African Bare-eyed Thrush
author,Rolf A. de By,James Bradley,James Bradley,James Bradley,James Bradley
license,Creative Commons Attribution-NonCommercial-ShareAlike 3.0,Creative Commons Attribution-NonCommercial-ShareAlike 4.0,Creative Commons Attribution-NonCommercial-ShareAlike 4.0,Creative Commons Attribution-NonCommercial-ShareAlike 4.0,Creative Commons Attribution-NonCommercial-ShareAlike 4.0
rating,4.0,3.5,3.5,5.0,4.5


## Audio Metadata

In [5]:
audio_metadata_df = pd.read_csv('audio_metadata.csv')
audio_metadata_df.head().T

,0,1,2,3,4
filename,abethr1/XC128013.ogg,abethr1/XC363501.ogg,abethr1/XC363502.ogg,abethr1/XC363503.ogg,abethr1/XC363504.ogg
sample_rate,32000,32000,32000,32000,32000
num_frames,1459513,597682,1240503,936229,1355024
num_channels,1,1,1,1,1
bits_per_sample,0,0,0,0,0
encoding,VORBIS,VORBIS,VORBIS,VORBIS,VORBIS
seconds,45.609781,18.677563,38.765719,29.257156,42.3445


# Audio Preprocessing

Here, we will preprocess audio and save new files. Some things to do:
1. Remove silence
2. Reduce sampling rate
3. Resize

In [6]:
prep_path = comp_path/'train_audio_prep-30s-32k'
prep_path.mkdir(exist_ok=True)

## Create Augmentation Pipeline

Initially, I'm not resizing. I want to see the distribution of sizes after removing silence before deciding what length of audio to use.

In [7]:
# cfg = AudioConfig.BasicSpectrogram()
# aud2spec = AudioToSpec.from_cfg(cfg)
resample = Resample(sr_new=32000)
rmsilence = RemoveSilence(remove_type=RemoveType.All, threshold=30)
pipe = Pipeline([resample, rmsilence])
# resize = ResizeSignal(duration=10e3)

## Preprocess

In [ ]:
preprocess_audio_folder(trn_path, output_dir=prep_path, pipeline=pipe)

  0%|          | 0/16941 [00:00<?, ?it/s]

## Check Audio Lengths

In [ ]:
# audio_prep_md_df = get_audio_files_metadata(get_audio_files(prep_path))
audio_prep_md_df = get_audio_files_metadata(trn_df.filename, parent=prep_path)

In [ ]:
_ = audio_prep_md_df.seconds.plot.hist(logy=True, bins=100)

In [ ]:
audio_prep_md_df.describe()

# References

[fastaudio]: Coultas Blum, Harry A, Lucas G. Scart, and Robert Bracco. 2020.
“Fastaudio.” *GitHub*. <https://github.com/fastaudio/fastaudio>.

# Further Exploration

* [TorchAudio](https://pytorch.org/audio/stable/feature_classifications.html)
* Use more of fastkaggle https://fastai.github.io/fastkaggle/
* FastAudio: https://colab.research.google.com/github/fastaudio/fastaudio/blob/master/docs/ESC50:%20Environmental%20Sound%20Classification.ipynb#scrollTo=6T8g4m4h3hNq

* One challenge with the dataset here is we have training audio of varying length with lots of quiet periods between bird calls. When we divide up a training audio signal, some pieces will not have bird calls and we don't want to assign the label(s) of the entire file to each piece. We could try multiple instance learning to deal with this. 

* Training audio with multiple labels may further complicate training. Can we start with multiple instance learning using only the training sets with single labels and then use that model to divide our multiple instances into groups with different labels for further training?

* Try TorchAudio pre-trained models as a backbone

* One-hot-encode extras? https://stackoverflow.com/questions/45312377/how-to-one-hot-encode-from-a-pandas-column-containing-a-list